Imports, paths, config

In [23]:
import json, pathlib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

# ── Paths
PROJECT_ROOT = pathlib.Path().resolve().parents[0]
RAW_FILE = PROJECT_ROOT / "data" / "raw" / "car.csv"
INTERIM_FILE = PROJECT_ROOT / "data" / "interim" / "cleaned.csv"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)

# ── Config
SEED = 42
TEST_SIZE = 0.20
TOP_OPTIONS = 10  # how many option flags to extract
TARGET = "Price($)"

CATEGORICAL_COLS = [
    "Brand","Model","Condition","FuelType","Transmission","DriveType",
    "BodyType","Color","Interior","City","AccidentHistory",
    "Insurance","RegistrationStatus"
]

NUMERIC_COLS = [
    "Year","Mileage(km)","EngineSize(L)","Horsepower","Torque",
    "Doors","Seats","FuelEfficiency(L/100km)","PricePerKm", TARGET
]

print("Project root:", PROJECT_ROOT)
print("Raw file:", RAW_FILE)

Project root: C:\Users\User\Desktop\Projects\Car_Price
Raw file: C:\Users\User\Desktop\Projects\Car_Price\data\raw\car.csv


Load & initial shape

In [24]:
df = pd.read_csv(RAW_FILE)
print("Initial shape:", df.shape)
df.head()


Initial shape: (50100, 25)


,Brand,Model,Year,CarAge,Condition,Mileage(km),EngineSize(L),FuelType,Horsepower,Torque,...,Color,Interior,Options,City,AccidentHistory,Insurance,RegistrationStatus,FuelEfficiency(L/100km),PricePerKm,Price($)
0,Porsche,Panamera,2008,17,Used,256395,3.3,Gasoline,513,395,...,White,Cloth,"Navigation, Cruise Control, Heated Seats, Blue...",Tehran,No,Valid,Incomplete,11.96,0.05,13884
1,Audi,A6,2023,2,Used,20433,2.2,Diesel,302,270,...,Black,Cloth,"Parking Sensors, Cruise Control, Touchscreen",Berlin,Yes,Expired,Incomplete,8.74,1.90,38888
2,BMW,X5,2022,3,Used,52328,3.2,Gasoline,400,388,...,Gray,Leather,"Touchscreen, Bluetooth, Cruise Control, Naviga...",Tokyo,Yes,Valid,Complete,15.68,0.63,33074
3,Hyundai,Tucson,2019,6,Used,91878,1.6,Hybrid,187,219,...,Silver,Cloth,"Sunroof, Rear Camera, Bluetooth, Parking Senso...",Delhi,No,Expired,Complete,9.45,0.14,12966
4,Fiat,500,2012,13,Damaged,192331,1.1,Gasoline,90,112,...,Red,Leather,"Heated Seats, Touchscreen",Delhi,No,Valid,Complete,7.16,0.01,2670


Drop exact duplicates

In [25]:
before = len(df)
df = df.drop_duplicates()
print(f"Removed duplicates: {before - len(df)} | Shape now: {df.shape}")

Removed duplicates: 100 | Shape now: (50000, 25)


Drop redundant columns (CarAge) and tidy strings

In [26]:
# 1) Drop CarAge (redundant with Year)
if "CarAge" in df.columns:
    df = df.drop(columns=["CarAge"])
    print("Dropped column: CarAge")

# 2) Strip whitespace from text columns
obj_cols = df.select_dtypes(include="object").columns.tolist()
for c in obj_cols:
    df[c] = df[c].astype(str).str.strip()
print("Stripped whitespace on object columns:", len(obj_cols))

Dropped column: CarAge
Stripped whitespace on object columns: 14


Coerce numeric columns safely

In [27]:
# Make sure numeric-like columns are numeric (coerce invalid to NaN)
coerced_report = {}
for c in NUMERIC_COLS:
    if c in df.columns:
        before_na = df[c].isna().sum()
        df[c] = pd.to_numeric(df[c], errors="coerce")
        after_na = df[c].isna().sum()
        coerced_report[c] = {"added_NA_from_coercion": int(after_na - before_na)}
pd.DataFrame(coerced_report).T

,added_NA_from_coercion
Year,0
Mileage(km),0
EngineSize(L),0
Horsepower,0
Torque,0
Doors,0
Seats,0
FuelEfficiency(L/100km),0
PricePerKm,0
Price($),0


Ensure target present, drop rows with missing target

In [28]:
if TARGET not in df.columns:
    raise ValueError(f"Target column {TARGET!r} not found in data.")
before = len(df)
df = df.dropna(subset=[TARGET])
print(f"Dropped rows with missing target: {before - len(df)} | Shape: {df.shape}")

Dropped rows with missing target: 0 | Shape: (50000, 24)


Missing value imputation

In [29]:
# Numeric → median ; Categorical → mode/Unknown
for c in NUMERIC_COLS:
    if c in df.columns:
        med = df[c].median()
        df[c] = df[c].fillna(med)

for c in CATEGORICAL_COLS:
    if c in df.columns:
        mode = df[c].mode(dropna=True)
        fill = mode.iloc[0] if not mode.empty else "Unknown"
        df[c] = df[c].fillna(fill)

Outlier handling

In [30]:
winsor_cols = [col for col in ["Price($)","Mileage(km)","Horsepower","EngineSize(L)","FuelEfficiency(L/100km)"] if col in df.columns]
winsor_cutoffs = {}
for c in winsor_cols:
    lo, hi = df[c].quantile([0.01, 0.99])
    df[c] = df[c].clip(lower=lo, upper=hi)
    winsor_cutoffs[c] = {"q01": float(lo), "q99": float(hi)}
winsor_cutoffs

{'Price($)': {'q01': 1492.99, 'q99': 85854.33000000007},
 'Mileage(km)': {'q01': 202.0, 'q99': 303929.03},
 'Horsepower': {'q01': 67.0, 'q99': 848.0},
 'EngineSize(L)': {'q01': 0.0, 'q99': 4.9},
 'FuelEfficiency(L/100km)': {'q01': 0.0, 'q99': 17.800100000000022}}

Feature engineering from Options

In [31]:
# Create OptionsCount + top-K option flags; then drop raw Options
from collections import Counter

top_options = []

if "Options" in df.columns:
    # Clean nulls
    df["Options"] = df["Options"].fillna("").astype(str)

    # 1) OptionsCount
    df["OptionsCount"] = df["Options"].apply(lambda s: len([t for t in s.split(",") if t.strip()]))

    # 2) Find top-K options (on the whole dataset is fine here since these are just presence flags)
    tokens = []
    for v in df["Options"]:
        tokens += [t.strip().lower() for t in v.split(",") if t.strip()]
    top_options = [opt for opt, _ in Counter(tokens).most_common(TOP_OPTIONS)]

    # 3) Build option-flag features
    for opt in top_options:
        colname = f"opt_{opt.replace(' ', '_')}"
        df[colname] = df["Options"].str.lower().apply(
            lambda s, o=opt: int(o in [t.strip() for t in s.split(",") if t.strip()])
        )

    # 4) Drop raw text column to keep dataset clean
    df = df.drop(columns=["Options"])
    print("Created OptionsCount +", len(top_options), "option flags; dropped raw 'Options' column.")

else:
    print("'Options' column not found; skipping option features.")


Created OptionsCount + 8 option flags; dropped raw 'Options' column.


In [32]:
# --- EXTRA PREPROCESSING CONFIG ---
TARGET = "Price($)"

RARE_THRESH = 50          # minimum count to keep a category (Brand/Model)
CORR_DROP_THRESHOLD = 0.95
LOG_FEATURES_AUTO = True  # auto-detect skewed numeric cols (excludes TARGET)
LOG_FEATURES_MANUAL = ["Mileage(km)"]  # add any you want to log1p (no target)

# where to record the changes
from pathlib import Path
import json
META_PATH = (PROJECT_ROOT / "data" / "processed" / "preprocess_meta.json")
with open(META_PATH) as f:
    meta = json.load(f)


In [33]:
def _norm(s):
    return (str(s).strip().lower().replace('-', ' ').replace('_',' '))

# Maps (extend if you see more variants in df.value_counts())
fuel_map = {
    "gasoline": "gasoline", "petrol": "gasoline", "benzine":"gasoline",
    "diesel": "diesel",
    "hybrid": "hybrid", "hybrid/electric": "hybrid",
    "electric": "electric", "ev":"electric"
}
trans_map = {"auto":"automatic","automatic":"automatic","manual":"manual","mt":"manual","at":"automatic"}
ins_map = {"valid":"valid","expired":"expired","no":"expired"}  # treat "no" as expired
reg_map = {"complete":"complete","incomplete":"incomplete"}
int_map = {"leather":"leather","cloth":"cloth","fabric":"cloth"}
# very simple color map to unify shades
basic_colors = {"white","black","silver","gray","grey","red","blue","green","yellow","brown","beige"}
def normalize_color(x):
    x = _norm(x)
    for c in basic_colors:
        if c in x:
            return "gray" if c=="grey" else c
    return "other"

# apply normalizations (only if column exists)
if "FuelType" in df.columns:
    df["FuelType"] = df["FuelType"].map(lambda x: fuel_map.get(_norm(x), _norm(x)))
if "Transmission" in df.columns:
    df["Transmission"] = df["Transmission"].map(lambda x: trans_map.get(_norm(x), _norm(x)))
if "Insurance" in df.columns:
    df["Insurance"] = df["Insurance"].map(lambda x: ins_map.get(_norm(x), _norm(x)))
if "RegistrationStatus" in df.columns:
    df["RegistrationStatus"] = df["RegistrationStatus"].map(lambda x: reg_map.get(_norm(x), _norm(x)))
if "Interior" in df.columns:
    df["Interior"] = df["Interior"].map(lambda x: int_map.get(_norm(x), _norm(x)))
if "Color" in df.columns:
    df["Color"] = df["Color"].map(normalize_color)

meta.setdefault("normalization", {})["applied"] = True


In [34]:
from collections import Counter

def group_rare(df, col, min_count, other_label="Other"):
    vc = Counter(df[col].astype(str))
    keep = {k for k,v in vc.items() if v >= min_count}
    df[col] = df[col].astype(str).apply(lambda x: x if x in keep else other_label)
    return sorted(list(keep))

rare_info = {}

if "Brand" in df.columns:
    kept_brands = group_rare(df, "Brand", RARE_THRESH, "Other")
    rare_info["Brand_min_count"] = RARE_THRESH
    rare_info["Brand_kept"] = kept_brands

# Model can be very granular—use a slightly lower threshold if you like
if "Model" in df.columns:
    kept_models = group_rare(df, "Model", max(20, RARE_THRESH//2), "Other")
    rare_info["Model_min_count"] = max(20, RARE_THRESH//2)
    rare_info["Model_kept"] = kept_models

meta["rare_category"] = rare_info


In [35]:
from scipy.stats import skew
logged_cols = set()

candidate_log = [c for c in df.select_dtypes(include=[np.number]).columns if c != TARGET]

if LOG_FEATURES_AUTO:
    for c in candidate_log:
        s = df[c].dropna()
        if (s <= 0).any():  # log1p needs >=0; shift if strictly negative (unlikely here)
            continue
        if abs(skew(s)) > 1.0:
            df[c] = np.log1p(df[c])
            logged_cols.add(c)

# plus any manual choices
for c in LOG_FEATURES_MANUAL:
    if c in df.columns and c != TARGET and (df[c] >= 0).all():
        if c not in logged_cols:
            df[c] = np.log1p(df[c])
            logged_cols.add(c)

meta["log_transform"] = {"columns": sorted(list(logged_cols)), "auto": LOG_FEATURES_AUTO}
print("Log-transformed columns:", sorted(list(logged_cols)))


Log-transformed columns: ['Doors', 'Horsepower', 'Mileage(km)']


In [36]:
with open(META_PATH, "w") as f:
    json.dump(meta, f, indent=2)

print("Extra preprocessing recorded to preprocess_meta.json")


Extra preprocessing recorded to preprocess_meta.json


Train/Test split

In [37]:
train_df, test_df = train_test_split(df, test_size=TEST_SIZE, random_state=SEED)
print("Train shape:", train_df.shape, " | Test shape:", test_df.shape)

Train shape: (40000, 32)  | Test shape: (10000, 32)


Save processed data + metadata

In [38]:
# Save an interim full-clean copy (pre-split)
INTERIM_FILE.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(INTERIM_FILE, index=False)

# Save final splits
train_path = PROCESSED_DIR / "train.csv"
test_path  = PROCESSED_DIR / "test.csv"
train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

# Save top options used for flags & winsor cutoffs (useful for report/app)
meta = {
    "top_options": top_options,
    "winsor_cutoffs": winsor_cutoffs,
    "seed": SEED,
    "test_size": TEST_SIZE,
}
with open(PROCESSED_DIR / "preprocess_meta.json", "w") as f:
    json.dump(meta, f, indent=2)

print("Saved files:")
print(" -", INTERIM_FILE)
print(" -", train_path)
print(" -", test_path)
print(" -", PROCESSED_DIR / "preprocess_meta.json")

Saved files:
 - C:\Users\User\Desktop\Projects\Car_Price\data\interim\cleaned.csv
 - C:\Users\User\Desktop\Projects\Car_Price\data\processed\train.csv
 - C:\Users\User\Desktop\Projects\Car_Price\data\processed\test.csv
 - C:\Users\User\Desktop\Projects\Car_Price\data\processed\preprocess_meta.json


Quick sanity report

In [39]:
report = {
    "rows_train": len(train_df),
    "rows_test": len(test_df),
    "dropped_columns": ["CarAge"] + (["PricePerKm"] if "PricePerKm" not in df.columns else []),
    "created_features": ["OptionsCount"] + [f"opt_{o.replace(' ','_')}" for o in top_options],
    "winsorized": list(winsor_cutoffs.keys()),
    "target": TARGET,
}
report

{'rows_train': 40000,
 'rows_test': 10000,
 'dropped_columns': ['CarAge'],
 'created_features': ['OptionsCount',
  'opt_cruise_control',
  'opt_navigation',
  'opt_heated_seats',
  'opt_parking_sensors',
  'opt_touchscreen',
  'opt_bluetooth',
  'opt_sunroof',
  'opt_rear_camera'],
 'winsorized': ['Price($)',
  'Mileage(km)',
  'Horsepower',
  'EngineSize(L)',
  'FuelEfficiency(L/100km)'],
 'target': 'Price($)'}

Final Check

In [40]:
print("Final dataset shape:", df.shape)

Final dataset shape: (50000, 32)


In [41]:
print("Missing values per column:\n", df.isna().sum())

Missing values per column:
 Brand                      0
Model                      0
Year                       0
Condition                  0
Mileage(km)                0
EngineSize(L)              0
FuelType                   0
Horsepower                 0
Torque                     0
Transmission               0
DriveType                  0
BodyType                   0
Doors                      0
Seats                      0
Color                      0
Interior                   0
City                       0
AccidentHistory            0
Insurance                  0
RegistrationStatus         0
FuelEfficiency(L/100km)    0
PricePerKm                 0
Price($)                   0
OptionsCount               0
opt_cruise_control         0
opt_navigation             0
opt_heated_seats           0
opt_parking_sensors        0
opt_touchscreen            0
opt_bluetooth              0
opt_sunroof                0
opt_rear_camera            0
dtype: int64


In [42]:
print("Duplicate rows:", df.duplicated().sum())

Duplicate rows: 0


In [43]:
print("Column types:\n", df.dtypes)

Column types:
 Brand                       object
Model                       object
Year                         int64
Condition                   object
Mileage(km)                float64
EngineSize(L)              float64
FuelType                    object
Horsepower                 float64
Torque                       int64
Transmission                object
DriveType                   object
BodyType                    object
Doors                      float64
Seats                        int64
Color                       object
Interior                    object
City                        object
AccidentHistory             object
Insurance                   object
RegistrationStatus          object
FuelEfficiency(L/100km)    float64
PricePerKm                 float64
Price($)                   float64
OptionsCount                 int64
opt_cruise_control           int64
opt_navigation               int64
opt_heated_seats             int64
opt_parking_sensors          int64
opt_t

In [44]:
df.head(10)

,Brand,Model,Year,Condition,Mileage(km),EngineSize(L),FuelType,Horsepower,Torque,Transmission,...,Price($),OptionsCount,opt_cruise_control,opt_navigation,opt_heated_seats,opt_parking_sensors,opt_touchscreen,opt_bluetooth,opt_sunroof,opt_rear_camera
0,Porsche,Panamera,2008,Used,12.454478,3.3,gasoline,6.242223,395,manual,...,13884.0,5,1,1,1,1,0,1,0,0
1,Audi,A6,2023,Used,9.924955,2.2,diesel,5.713733,270,manual,...,38888.0,3,1,0,0,1,1,0,0,0
2,BMW,X5,2022,Used,10.865306,3.2,gasoline,5.993961,388,automatic,...,33074.0,5,1,1,0,1,1,1,0,0
3,Hyundai,Tucson,2019,Used,11.428228,1.6,hybrid,5.236442,219,automatic,...,12966.0,5,0,1,0,1,0,1,1,1
4,Fiat,500,2012,Damaged,12.166978,1.1,gasoline,4.510860,112,automatic,...,2670.0,2,0,0,1,0,1,0,0,0
5,Porsche,911 Carrera,2018,Used,11.617006,3.3,gasoline,6.338594,392,automatic,...,47830.0,1,0,0,0,0,0,0,1,0
6,Mercedes-Benz,S-Class,2019,Used,11.321862,4.9,hybrid,6.234411,750,manual,...,51189.0,3,0,1,0,1,1,0,0,0
7,Porsche,Panamera,2014,Used,12.001965,3.5,gasoline,6.242223,428,manual,...,27296.0,5,1,0,1,0,1,1,1,0
8,Audi,Q7,2007,Used,12.522605,2.0,gasoline,5.755742,277,automatic,...,5389.0,2,0,0,0,0,1,1,0,0
9,Ford,Mustang,2019,Used,11.466871,4.6,gasoline,5.899897,588,manual,...,10704.0,5,1,1,0,1,1,1,0,0
